In [22]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index,d_hkl,three_dot,cross_plane, get_right_hand
from csl_generator import print_list, getsigmas, get_theta_m_n_list
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot

from Element import Element

In [23]:
from abc import ABCMeta, abstractclassmethod
import copy

In [24]:
def get_expansion_xyz(cell,cell_size):
    cell_x, cell_y, cell_z = cell_size
    exp_x = ceil(cell_x/norm(cell[:,0]))
    exp_y = ceil(cell_y/norm(cell[:,1]))
    exp_z = ceil(cell_z/norm(cell[:,2]))
    return exp_x, exp_y, exp_z

In [25]:
def getLatE(dirname):
    """
    read the Energy of single crystal 
    """
    with open(f'{dirname}/atomsout','r') as f:
        lines=f.readlines()
        LatE = lines[-4:][0].split()[4].replace(';', '')
    return float(LatE)

def getLatP(dirname):
    """
    read the Lattice Parameter
    """
    with open(f'{dirname}/atomsout','r') as f:
        lines=f.readlines()
        LatP = lines[-5:][0].split()[4].replace(';', '')
    return float(LatP)

def get_mass(dirname):
    with open(f'{dirname}/mass','r') as f:
        lines=f.read()
    return float(lines)

In [26]:
class Element2:
    def __init__(self,name,dirname):
        self.name = name
        self.dirname = dirname
        self.grandenergy = getLatE(dirname)
        self.LP = getLatP(dirname)
        self.mass = get_mass(dirname)
        self.ciffile = glob.glob(f"{dirname}/*.cif")[0]
        with open(f"{self.dirname}/proto.in","r") as f:
            self.proto_text = f.read() 
    
    def potential(self):
        for dirs in glob.glob(f"{self.dirname}/potential/*"):
            shutil.copy(f"{dirs}",f"{os.getcwd()}")
        
    def proto(self):
        shutil.copy(f"{self.dirname}/proto.in",f"{os.getcwd()}")

In [27]:
class ProtoinCreator(metaclass=ABCMeta):
    # element →　Element クラス　オブジェクト　インスタンス化"前"
    # SanplingMethod　→　SamplingMethodCreator クラス　オブジェクト　インスタンス化"前"
    def __init__(self,element,SamplingMethod):
        self.element = element
        self.sampling_method = SamplingMethod
    def generate(self,axis,hkl):
        self.body_text = ProtoinBodyCreator(self.sampling_method(axis,hkl)).generate()
    # サブクラスがインスタンス化されるまで保留して一旦設計を進めることができる
    
    @abstractclassmethod
    def header(self):
        pass
    
    @abstractclassmethod
    def body(self):
        pass 
    
    
    #テンプレートの役割を持つクラス
    def create(self):
        h = self.header()
        b = self.body()
        return f"{h}\n\n{b}"

In [28]:
class Protoin(ProtoinCreator):
    
    # 継承先を固定しておけばテンプレートを動かさなくていい
    def header(self):
        text = f"""


variable element_mass equal {self.element.mass} 

variable minimumenergy equal {self.element.grandenergy}

variable lattice_param equal {self.element.LP}
                """
        
        return text

    
    def body(self):
        return self.body_text
    

In [29]:
class ProtoinBodyCreator:
    def __init__(self,sampling_method_creator):
        self.sampling = sampling_method_creator
        
    def generate(self):
        return self.sampling.calc_condition
    ###ここを変える

In [30]:
class Calc_Condition:
    def __init__(self,element,SamplingMethod,cell_size,fix_bulk_region,middle_region,start_bulk,end_bulk,cnid_sampling):

        self.cnid_sampling = cnid_sampling
        self.element = element
        self.protoin = Protoin(self.element,SamplingMethod)
        self.cell_size = cell_size
        self.fix_bulk_region = fix_bulk_region
        self.middle_region = middle_region
        self.start_bulk = start_bulk
        self.end_bulk = end_bulk
        
    def define_bicrystal_regions(self,xhi):
        tol = 1e-5
        """
        generate a file defining some regions in the LAMMPS and define the atoms
        inside these regions into some groups.
        argument:
        region_names --- list of name of regions
        region_los --- list of the low bounds
        region_his --- list of the hi bounds
        """
        end_fixbulk1 = xhi/2-self.fix_bulk_region
        start_fixbulk2 = xhi/2+self.fix_bulk_region
        start_middle = xhi/2-self.middle_region
        end_middle = xhi/2+self.middle_region
        start_right = xhi/2 - tol
        start_bulk = self.start_bulk
        end_bulk = self.end_bulk


        with open('blockfile', 'w') as fb:
            fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(end_fixbulk1))
            fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_fixbulk2))
            fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_middle,end_middle))
            fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_bulk,end_bulk))
            fb.write('group fixbulk1 region fixbulk1 \n')
            fb.write('group fixbulk2 region fixbulk2 \n')
            fb.write('group middle region middle \n')
            fb.write('group right region right \n')
            fb.write("group left region left \n")
            fb.write('group bulk region bulk \n')
        
    def define_bicrystal_regions2(self,xhi):
        half_cell = xhi/2
        text = f"""

    displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

    group left_dash region left 

    group deleted_atoms subtract left_dash left

    displace_atoms deleted_atoms move {half_cell} 0 0 units box

    group fixbulk1_dash region fixbulk1 
    group fixbulk2_dash region fixbulk2 
    group middle_dash region middle 
    group bulk_dash region bulk
                """
        with open("blockfile2", "w") as f:
            f.writelines(text)
        
    def generate_files(self,axis,hkl,xhi):
        self.protoin.generate(axis,hkl)
        self.element.potential()
        self.define_bicrystal_regions(xhi)
        self.define_bicrystal_regions2(xhi)
        
        if self.cnid_sampling==True:
            self.element.proto()
        else:
            with open("proto.in","w") as f:
                f.writelines(self.protoin.create())

In [31]:
class core2(core):
    def __init__(self, file_1, file_2):
        super().__init__(file_1,file_2)
    def compute_stgb(self, axis, plane_n, lim = 20, tol = 1e-10):
        """
        compute the transformation to obtain the supercell of the two slabs forming a interface
        argument:
        hkl --- miller indices of the plane expressed in lattice_1
        lim --- the limit searching for a CSL vector cross the plane
        normal_ortho --- whether limit the vector crossing the GB to be normal to the GB
        plane_ortho --- whether limit the two vectors in the GB plane to be orthogonal
        tol --- tolerance judging whether orthogonal
        """
        hkl = MID(self.lattice_1,plane_n)
        self.d1 = d_hkl(self.lattice_1, hkl)
        lattice_2 = three_dot(self.R, self.D, self.lattice_2)
        hkl_2 = get_primitive_hkl(hkl, self.lattice_1, lattice_2)
        self.d2 = d_hkl(lattice_2, hkl_2)
        hkl_c = get_primitive_hkl(hkl, self.lattice_1, self.CSL) # miller indices of the plane in CSL
        hkl_c = np.array(hkl_c)
        
        #plane_n = dot(self.lattice_1,hkl)
        axis_cartesian = dot(self.conv_lattice_1,axis)
        v1 = cross_plane(lattice=self.CSL, n=plane_n, lim=lim,  orthogonal=True, tol=tol) # a CSL basic vector cross the plane
        v2 = cross_plane(lattice=self.CSL, n=axis_cartesian, lim=lim, orthogonal=True, tol=tol)
        v3_cartesian = cross(v1,v2)
        v3 = cross_plane(lattice=self.CSL, n=v3_cartesian, lim=lim, orthogonal=True, tol=tol)
        supercell = np.column_stack((v1,v2,v3)) # supercell of the bicrystal
        supercell = get_right_hand(supercell) # check right-handed


        self.bicrystal_U1 = np.array(np.round(dot(inv(self.lattice_1), supercell),8),dtype = int)
        self.bicrystal_U2 = np.array(np.round(dot(inv(self.lattice_2_TD), supercell),8),dtype = int)
        self.cell_calc.compute_CNID(hkl)
        CNID = self.cell_calc.CNID
        self.CNID = dot(self.lattice_1, CNID)

In [32]:
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

In [33]:
def get_thetas(axis,max_sigma):
    theta_list = []
    sigma_list = []
    lists, thetas = getsigmas(axis, max_sigma)
    return thetas

In [34]:
def get_odd_num(num):
    if num%2==1:
        return num
    num = int(num/2)
    return get_odd_num(num)

In [35]:
def get_gb_files(interface,axis,hkl,calc_condition,input_dirname=None):
    interface.compute_stgb(axis, hkl, lim = 10, tol = 1e-5)
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
    x,y,z = get_expansion_xyz(half_cell,calc_condition.cell_size)
    xhi = 2*x*norm(half_cell[:,0])
    
#     hkl_name = get_primitive_hkl(hkl, interface.lattice_1, interface.conv_lattice_1)
    hkl_name = np.array(np.round(dot(inv(interface.conv_lattice_1), hkl),8),dtype = int)
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    axis_x, axis_y, axis_z = axis
    dirname = f"{axis_x}{axis_y}{axis_z}_{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}_gb"
    if not input_dirname==None:
        dirname = input_dirname
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname) 
    
    if (axis == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = True)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)
        
    CNID = dot(interface.orient, interface.CNID)
    v1 = np.array([0,1.,0])*CNID[:,0][1] + np.array([0,0,1.])*CNID[:,0][2]
    v2 = np.array([0,1.,0])*CNID[:,1][1] + np.array([0,0,1.])*CNID[:,1][2]

    n1 = int(ceil(norm(v1)/0.2))
    n2 = int(ceil(norm(v2)/0.2))
    write_trans_file(v1,v2,n1,n2)

    calc_condition.generate_files(axis=axis,hkl=hkl_name,xhi=xhi)


    os.chdir(os.pardir)

In [36]:
def get_all_STGBs(axis,calc_condition,max_sigma):

    #cif_filename = glob.glob(f"/homenfs1/yhata/python_script/{element.element}/*.cif")[0]

    cif_filename = calc_condition.element.ciffile
    LP = calc_condition.element.LP

    my_interface = core2(cif_filename, cif_filename)
    my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = LP/(norm(my_interface.conv_lattice_1[:,0]))
    my_interface.lattice_1 =  my_interface.lattice_1*factor
    my_interface.lattice_2 =  my_interface.lattice_2*factor
    my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
    my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
    my_interface.lattice_1 = my_interface.conv_lattice_1
    my_interface.lattice_2 = my_interface.conv_lattice_2

    my_interface2 = core2(cif_filename, cif_filename)
    my_interface2.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = LP/(norm(my_interface2.conv_lattice_1[:,0]))
    my_interface2.lattice_1 =  my_interface2.lattice_1*factor
    my_interface2.lattice_2 =  my_interface2.lattice_2*factor
    my_interface2.conv_lattice_1 =  my_interface2.conv_lattice_1*factor
    my_interface2.conv_lattice_2 =  my_interface2.conv_lattice_2*factor
    axis_cart = dot(my_interface2.conv_lattice_1, np.array(axis))

    axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)

    thetas = get_thetas(axis,max_sigma)


    for theta in thetas:
        
        my_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
        my_interface2.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        n_1, n_2 = get_a_b(my_interface.CSL, dot(my_interface.lattice_1,np.array(axis)))
        
        get_gb_files(interface=my_interface2,axis=axis,hkl=n_1,calc_condition=calc_condition)
        get_gb_files(interface=my_interface2,axis=axis,hkl=n_2,calc_condition=calc_condition)

In [37]:
  class MyInterface:
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        
        self.my_interface = core2(self.cif_filename, self.cif_filename)
        
        self.my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=self.max_sigma, sgm2=self.max_sigma, dd = 1e-4)
        factor = self.LP/(norm(self.my_interface.conv_lattice_1[:,0]))
        self.my_interface.lattice_1 =  self.my_interface.lattice_1*factor
        self.my_interface.lattice_2 =  self.my_interface.lattice_2*factor
        self.my_interface.conv_lattice_1 =  self.my_interface.conv_lattice_1*factor
        self.my_interface.conv_lattice_2 =  self.my_interface.conv_lattice_2*factor
        

In [38]:
class Factory(object): # ここはインスタンスを生成する工場だから普段インスタンスを作ることをやる
    def create(self,hkl): # インスタンスをcreate
        # 回転軸、回転角、plane_nを受け取って作成
        axis, theta, plane_n = self.get_gb_info_from_hkl(hkl)
        self.createGB(axis,theta,plane_n)
    
    def createGB(self,axis,theta,plane_n):
        # interfaceインスタンス化 + get_gbfiles実行
        pass

    
    def generate_hkl_list(self,axis,max_sigma):
        pass
    
    def get_gb_info_from_hkl(self,hkl):
        pass
    


In [39]:
class GBFactory(Factory):
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        self.primitive_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        self.conventional_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        
    def createGB(self,axis,theta,plane_n):
    # interface serch csl create_gbfiles実行
        axis_cart = dot(self.primitive_interface.conv_lattice_1, np.array(axis))

        axis_pri = np.array(np.round(dot(inv(self.primitive_interface.lattice_1), axis_cart),8),dtype = int)
        print(axis_pri)
        print(theta)
        print(self.primitive_interface.lattice_1)
        
        self.primitive_interface.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        get_gb_files(self.primitive_interface,axis,plane_n,self.calc_condition)
    
    @classmethod
    def generate_hkl_list(cls,axis,max_sigma,calc_condition):
        conventional_interface = copy.deepcopy(cls(max_sigma,axis,calc_condition).conventional_interface)
        conventional_interface.lattice_1 = conventional_interface.conv_lattice_1
        conventional_interface.lattice_2 = conventional_interface.conv_lattice_2
        
        thetas = get_thetas(axis,max_sigma)

        gb_dict = {}

        for theta in thetas:

            conventional_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
            n_1, n_2 = get_a_b(conventional_interface.CSL, dot(conventional_interface.lattice_1,np.array(axis)))

            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_1),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info1 = {f"{hkl_x}{hkl_y}{hkl_z}":[axis,theta,n_1]}

            hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_2),8),dtype = int)
            hkl_name = np.array(hkl_name, dtype=int)
            hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
            hkl_x, hkl_y, hkl_z = hkl_name
            axis_x, axis_y, axis_z = axis

            gb_info2 = {f"{hkl_x}{hkl_y}{hkl_z}":[axis,theta,n_2]}

            gb_dict = {**gb_dict,**gb_info1,**gb_info2}
        
        cls.gb_dict = gb_dict
        
        return cls(max_sigma,axis,calc_condition)
    
    def get_gb_info_from_hkl(self,hkl):
        return self.gb_dict[hkl]
     
    @staticmethod
    def get_thetas(axis,max_sigma):
        theta_list = []
        sigma_list = []
        lists, thetas = getsigmas(axis, max_sigma)
        return thetas

In [40]:
class SamplingMethodCreator_Fe_deletelayer:
    def __init__(self,axis,hkl):
        self.squared_hkl = np.square(hkl).sum()
        print(self.squared_hkl)
        self.sigma = get_odd_num(self.squared_hkl)
        x, y, z = hkl
        if ((x + y + z)%2==1):
            self.one_two = 2
        else:
            self.one_two = 1
        self.calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two}

variable n_layer equal {self.sigma}

variable norm_hkl equal sqrt(v_squared_hkl) 

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Fe Fe.meam Fe
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # Fe

include ./blockfile2


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump            1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle_dash)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle_dash))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle_dash)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x


#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{tx}} ${{gbemJm2}}" append results

#then go to the outter loop
next a
jump proto.in loopa
        """
        

In [41]:
class SamplingMethodCreator_cnid:
    def __init__(self,axis,hkl):
        self.calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu 
neighbor 2.0 bin 
neigh_modify delay 10 check yes 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{ave_dis_x}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """

In [105]:
class SamplingMethodCreator_MLIP:
    def __init__(self,axis,hkl):
        self.calc_condition = f"""
# define looping parameter------------------ 

clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style      polymlp
pair_coeff      * * mlp.lammps Cu

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump 		1 all custom 1 final id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x

#----------- output calculation result of each loop into results file 
print "${{gbemJm2}} ${{ave_dis_x}}" append results

        """

In [126]:
class SamplingMethodCreator_deletelayer_MLIP:
    def __init__(self,axis,hkl):
        self.squared_hkl = np.square(hkl).sum()
        print(self.squared_hkl)
        self.sigma = get_odd_num(self.squared_hkl)
        x, y, z = hkl
        if ((x%2==1) & (y%2==1) & (z%2==1)):
            self.one_two = 1    
        else:
            self.one_two = 2
        self.calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two}

variable n_layer equal {self.sigma}

variable norm_hkl equal sqrt(v_squared_hkl) 

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style      polymlp
pair_coeff      * * mlp.lammps Cu

mass 1 ${{element_mass}} # Cu

include ./blockfile2


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump            1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle_dash)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle_dash))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle_dash)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x


#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{tx}} ${{gbemJm2}}" append results

#then go to the outter loop
next a
jump proto.in loopa
        """
        

In [98]:
class SamplingMethodCreator_MEAM:
    def __init__(self,axis,hkl):
        self.calc_condition = f"""
# define looping parameter------------------ 

clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu 
neighbor 2.0 bin 
neigh_modify delay 10 check yes 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump 		1 all custom 1 final id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x

#----------- output calculation result of each loop into results file 
print "${{gbemJm2}} ${{ave_dis_x}}" append results

        """

In [141]:
class SamplingMethodCreator_cnid_MLIP:
    def __init__(self,axis,hkl):
        self.calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Cu

# ---------- Define Interatomic Potential --------------------- 
pair_style      polymlp
pair_coeff      * * mlp.lammps Cu

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{ave_dis_x}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """

In [162]:
class SamplingMethodCreator_deletelayer_MEAM:
    def __init__(self,axis,hkl):
        self.squared_hkl = np.square(hkl).sum()
        print(self.squared_hkl)
        self.sigma = get_odd_num(self.squared_hkl)
        x, y, z = hkl
        if ((x%2==1) & (y%2==1) & (z%2==1)):
            self.one_two = 1    
        else:
            self.one_two = 2
        self.calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two}

variable n_layer equal {self.sigma}

variable norm_hkl equal sqrt(v_squared_hkl) 

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam Cu Cu.meam Cu 
neighbor 2.0 bin 
neigh_modify delay 10 check yes 

mass 1 ${{element_mass}} # Cu

include ./blockfile2


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump            1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle_dash)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle_dash))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle_dash)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x


#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{tx}} ${{gbemJm2}}" append results

#then go to the outter loop
next a
jump proto.in loopa
        """
        

In [178]:
class SamplingMethodCreator_deletelayer_MLIP_Mo:
    def __init__(self,axis,hkl):
        self.squared_hkl = np.square(hkl).sum()
        print(self.squared_hkl)
        self.sigma = get_odd_num(self.squared_hkl)
        x, y, z = hkl
        if ((x + y + z)%2==1):
            self.one_two = 2
        else:
            self.one_two = 1
        self.calc_condition = f"""
variable squared_hkl equal {self.squared_hkl}

variable one_two equal {self.one_two}

variable n_layer equal {self.sigma}

variable norm_hkl equal sqrt(v_squared_hkl) 

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_hkl) / (v_squared_hkl)) / (v_one_two) * v_lattice_param"

variable tx equal "(-v_dx*(v_a-1))" # displacement in x direction
variable ty equal 0 # displacement in y direction 
variable tz equal 0 # displacement in z direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style      polymlp
pair_coeff      * * mlp.lammps Mo

mass 1 ${{element_mass}} # Mo

include ./blockfile2


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump            1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle_dash)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle_dash))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle_dash)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x


#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{tx}} ${{gbemJm2}}" append results

#then go to the outter loop
next a
jump proto.in loopa
        """
        

In [179]:
class SamplingMethodCreator_cnid_MLIP_Mo:
    def __init__(self,axis,hkl):
        self.calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # a label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 
# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

mass 1 ${{element_mass}} # Mo

# ---------- Define Interatomic Potential --------------------- 
pair_style      polymlp
pair_coeff      * * mlp.lammps Mo

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in Cu crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL 0.0 0.0

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{ave_dis_x}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """

使用する元素をポテンシャルを含めて定義

In [50]:
element = "Fe"
initializing_dir = f"/homenfs1/yhata/calc/220516/{element}"
Fe = Element2(element,initializing_dir)

In [163]:
element = "Cu"
initializing_dir = f"/homenfs1/yhata/calc/220516/{element}"
Cu = Element2(element,initializing_dir)

In [146]:
element = "Cu"
initializing_dir = f"/homenfs1/yhata/calc/220523/potentials/pair_37"
Cu = Element2(element,initializing_dir)

In [215]:
element = "Mo"
initializing_dir = f"/homenfs1/yhata/calc/220524/Mo_potentials/gtinv-215"
Mo = Element2(element,initializing_dir)

サンプリングの方法を決定する

In [107]:
Samplingmethod = SamplingMethodCreator_MLIP

In [99]:
Samplingmethod = SamplingMethodCreator_MEAM

In [128]:
Samplingmethod = SamplingMethodCreator_deletelayer_MLIP

In [147]:
Samplingmethod = SamplingMethodCreator_cnid_MLIP

In [164]:
Samplingmethod = SamplingMethodCreator_deletelayer_MEAM

In [216]:
Samplingmethod = SamplingMethodCreator_deletelayer_MLIP_Mo

In [201]:
# Samplingmethod = SamplingMethodCreator_cnid_MLIP_Mo

計算条件を決定する

In [217]:
fix_bulk_region = 30
middle_region = 50
start_bulk = 160
end_bulk = 165
cell_size = [100,1,1]

まとめてcalc_conditionオブジェクトを作成する

In [166]:
calc_condition = Calc_Condition(element=Cu,SamplingMethod=Samplingmethod,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk,cnid_sampling=False)


In [218]:
calc_condition = Calc_Condition(element=Mo,SamplingMethod=Samplingmethod,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk,cnid_sampling=False)

回転軸と最大Σ値を指定

In [219]:
max_sigma = 100
axis = [1,1,0]

指定した回転軸に関するmax_sigmaより小さい粒界のリストを作成

In [220]:
gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.15047266e+00 3.15047266e+00 3.85821625e-16]
rotation matrix = [[ 0.66666667  0.33333333  0.66666667]
 [ 0.33333333  0.66666667 -0.66666667]
 [-0.66666667  0.66666667  0.33333333]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  1]
 [ 1 -1 -1]
 [ 0  1 -1]]; sigma_1 = 3

U2 = 
[[ 1 -1  1]
 [ 1  0 -1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.15047266e+00 3.15047266e+00 3.85821625e-16] ; theta = 70.52877927

[3.15047266e+00 3.15047266e+00 3.85821625e-16]
rotation matrix = [[ 0.88888889  0.11111111  0.44444444]
 [ 0.11111111  0.88888889 -0.44444444]
 [-0.44444444  0.44444444  0.77777778]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -2]
 [ 1  0  2]
 [ 0 -2  1]]; sigma_1 = 9

U2 = 
[[ 1  0 -2]
 [ 1 -1  2]
 [ 0 -2 -1]]; sigma_2 = 9

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.15047266e+00 

rotation matrix = [[ 0.98989899  0.01010101  0.14141414]
 [ 0.01010101  0.98989899 -0.14141414]
 [-0.14141414  0.14141414  0.97979798]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0 -7]
 [ 1  1  7]
 [ 0 -7  1]]; sigma_1 = 99

U2 = 
[[ 1  1 -7]
 [ 1  0  7]
 [ 0 -7 -1]]; sigma_2 = 99

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.15047266e+00 3.15047266e+00 3.85821625e-16] ; theta = 11.53636227

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!


見つけた粒界(conventional cellで)のリストを粒界面方位(hkl)で見る

In [221]:
gbfactory.gb_dict.keys()

dict_keys(['111', '211', '221', '411', '311', '332', '322', '433', '331', '611', '511', '552', '441', '811', '443', '833', '533', '655', '551', '1011', '722', '774', '553', '1033', '733', '776', '661', '1211', '744', '877', '911', '992', '922', '994', '665', '1255', '771', '1411'])

その中から作りたい粒界面指数を選んでlammpsのシミュレーションに使うファイルを作成

In [222]:
gbfactory.create("332")

[2 1 1]
50.478803641357835
[[ 1.57523633  1.57523633 -1.57523633]
 [ 1.57523633 -1.57523633  1.57523633]
 [ 1.57523633 -1.57523633 -1.57523633]]
[3.15047266e+00 3.15047266e+00 4.44089210e-16]
rotation matrix = [[ 0.81818182  0.18181818  0.54545454]
 [ 0.18181818  0.81818182 -0.54545454]
 [-0.54545454  0.54545454  0.63636364]]
Congrates, we found an appx CSL!

U1 = 
[[ 2  0  1]
 [ 1  1 -2]
 [ 1  2  1]]; sigma_1 = 11

U2 = 
[[2 0 1]
 [1 2 0]
 [1 1 3]]; sigma_2 = 11

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.15047266e+00 3.15047266e+00 4.44089210e-16] ; theta = 50.47880354

[-9.45141797  9.45141797  6.30094531]
[-3.15047266e+00 -3.15047266e+00 -4.44089210e-16]
[ 1.57523633 -1.57523633  4.72570898]
[[ 0 -2  0]
 [-5 -1 -1]
 [ 1 -1 -2]]
[[ 0 -2  0]
 [-1 -1 -2]
 [ 5 -1 -1]]
num of atoms1 = 22
[[ 1.03439186e+02 -6.42363156e-16  1.98273947e-16]
 [-4.35587817e-15  4.45544116e+00 -1.00609615e-16]
 [-6.58123513e-15 -3.10737765e-16  5.22446786e+00]]
22
